In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

In [2]:
from ib_insync import *
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import math
import datetime 
import statsmodels.formula.api as smf 
from datetime import date, time, datetime, timedelta
import seaborn as sns
import random
from ta import add_all_ta_features
from ta.utils import dropna
from ta.trend import *

In [3]:
#define a function to get IB data. endDate is the last date of the historical data 

def get_data(contract, history, freq, side, endDate =''):
    bar = ib.reqHistoricalData(
        contract,
        endDateTime = endDate,
        durationStr=history,
        barSizeSetting= freq,
        whatToShow=side,
        useRTH=True,
        formatDate=1)
    return util.df(bar)

history = '6 Y'
freq = '1 day'
side ='Trades'

In [59]:
# Pull data from ADRs in "Asian_ADRs.csv"
adr_symbol = "IMMP"
adr_exchange = 'SMART'
adr_currency = "USD"

underlying_symbol = 'IMM'
underlying_exchange = 'SMART'
underlying_currency = 'AUD'
underlying_country = "Australia"

folder_name = underlying_country + "/" + adr_symbol + "_" + underlying_symbol + "/"

In [40]:
adr_contract = Contract(symbol = adr_symbol, secType = 'STK', exchange = adr_exchange, currency = adr_currency)
ib.qualifyContracts(adr_contract)
adr_df = get_data(adr_contract, history, freq, side)
print(adr_df.head(3))
adr_df.to_csv(folder_name + "adr.csv")

         date   open   high    low  close  volume  average  barCount
0  2015-04-13  50.86  50.86  48.36  48.88    4230  49.4060      2563
1  2015-04-14  47.16  47.74  46.50  46.91    2403  47.0195      1430
2  2015-04-15  46.91  47.18  46.43  46.59    1146  46.8090       854


In [60]:
underlying_contract = Contract(symbol = underlying_symbol, secType = 'STK', exchange = underlying_exchange, currency = underlying_currency)
ib.qualifyContracts(underlying_contract)
underlying_df = get_data(underlying_contract, history, freq, side)
print(underlying_df.head(3))
underlying_df.to_csv(folder_name + "underlying.csv")

Error 162, reqId 63: Historical Market Data Service error message:No market data permissions for ASX STK, contract: Contract(secType='STK', conId=298064707, symbol='IMM', exchange='SMART', primaryExchange='ASX', currency='AUD', localSymbol='IMM', tradingClass='IMM')


AttributeError: 'NoneType' object has no attribute 'head'

In [1]:
# No underlying data for:
# VEDL-VEDL
# PLL-PLL
# IMMP-IMM
# KZIA-KZA
# ATHE-ATH

In [61]:
underlying_contract = Contract(symbol = underlying_symbol, secType = 'STK', exchange = underlying_exchange, currency = underlying_currency)
ib.qualifyContracts(underlying_contract)

[Contract(secType='STK', conId=298064707, symbol='IMM', exchange='SMART', primaryExchange='ASX', currency='AUD', localSymbol='IMM', tradingClass='IMM')]

In [62]:
underlying_df = get_data(underlying_contract, history, freq, side)
print(underlying_df.head(3))
#underlying_df.to_csv(folder_name + "underlying.csv")

Error 162, reqId 65: Historical Market Data Service error message:No market data permissions for ASX STK, contract: Contract(secType='STK', conId=298064707, symbol='IMM', exchange='SMART', primaryExchange='ASX', currency='AUD', localSymbol='IMM', tradingClass='IMM')


AttributeError: 'NoneType' object has no attribute 'head'